In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob



Load Dataset


In [11]:
df=pd.read_csv('medical_insurance.csv')
df.head()

,person_id,age,sex,region,urban_rural,income,education,marital_status,employment_status,household_size,...,liver_disease,arthritis,mental_health,proc_imaging_count,proc_surgery_count,proc_physio_count,proc_consult_count,proc_lab_count,is_high_risk,had_major_procedure
0,75722,52,Female,North,Suburban,22700,Doctorate,Married,Retired,3,...,0,1,0,1,0,2,0,1,0,0
1,80185,79,Female,North,Urban,12800,No HS,Married,Employed,3,...,0,1,1,0,0,1,0,1,1,0
2,19865,68,Male,North,Rural,40700,HS,Married,Retired,5,...,0,0,1,1,0,2,1,0,1,0
3,76700,15,Male,North,Suburban,15600,Some College,Married,Self-employed,5,...,0,0,0,1,0,0,1,0,0,0
4,92992,53,Male,Central,Suburban,89600,Doctorate,Married,Self-employed,2,...,0,1,0,2,0,1,1,0,1,0


In [12]:
print(f"Dataset Shape: {df.shape}")
print(f"\nFirst Few Rows:")
print(df.head())
print(f"\nData Types:")
print(df.dtypes)
print(f"\nMissing Values:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values")
print(f"\nBasic Statistics:")
print(df.describe())

Dataset Shape: (100000, 54)

First Few Rows:
   person_id  age     sex   region urban_rural  income     education  \
0      75722   52  Female    North    Suburban   22700     Doctorate   
1      80185   79  Female    North       Urban   12800         No HS   
2      19865   68    Male    North       Rural   40700            HS   
3      76700   15    Male    North    Suburban   15600  Some College   
4      92992   53    Male  Central    Suburban   89600     Doctorate   

  marital_status employment_status  household_size  ...  liver_disease  \
0        Married           Retired               3  ...              0   
1        Married          Employed               3  ...              0   
2        Married           Retired               5  ...              0   
3        Married     Self-employed               5  ...              0   
4        Married     Self-employed               2  ...              0   

   arthritis mental_health proc_imaging_count  proc_surgery_count  \
0       

Data Preprocessing


In [14]:
df.isnull().sum()


person_id                          0
age                                0
sex                                0
region                             0
urban_rural                        0
income                             0
education                          0
marital_status                     0
employment_status                  0
household_size                     0
dependents                         0
bmi                                0
smoker                             0
alcohol_freq                   30083
visits_last_year                   0
hospitalizations_last_3yrs         0
days_hospitalized_last_3yrs        0
medication_count                   0
systolic_bp                        0
diastolic_bp                       0
ldl                                0
hba1c                              0
plan_type                          0
network_tier                       0
deductible                         0
copay                              0
policy_term_years                  0
p

In [ ]:
df['alcohol_freq'].fillna('Occasional', inplace=True)


Unique values in alcohol_freq:
alcohol_freq
Occasional    45078
Weekly        19833
Daily          5006
Name: count, dtype: int64

Null values: 30083


In [16]:
# Fill null values in alcohol_freq with the mode (most frequent value)
mode_value = df['alcohol_freq'].mode()[0]
df['alcohol_freq'].fillna(mode_value, inplace=True)

print(f"Filled null values with: {mode_value}")
print(f"Remaining null values: {df['alcohol_freq'].isnull().sum()}")
print(f"\nUpdated value counts:")
print(df['alcohol_freq'].value_counts())

Filled null values with: Occasional
Remaining null values: 0

Updated value counts:
alcohol_freq
Occasional    75161
Weekly        19833
Daily          5006
Name: count, dtype: int64


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23212\813491754.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['alcohol_freq'].fillna(mode_value, inplace=True)


In [17]:
# Verify no null values remain in the entire dataset
print("Total null values in dataset:")
null_check = df.isnull().sum()
print(f"Total: {null_check.sum()}")
print(f"\nDataset shape: {df.shape}")

Total null values in dataset:
Total: 0

Dataset shape: (100000, 54)
